# 记录一些carla中actor的基本操作
    在 CARLA 中，Actor 是模拟环境中执行动作的元素，它们可以影响其他 Actor。
    CARLA 中的 Actor 包括车辆（vehicles）、行人（walkers）、传感器（sensors）、交通标志（traffic signs）、交通信号灯（traffic lights）以及观察者（spectator）。
    充分了解如何操作它们至关重要。本节将介绍 Actor 的生成、销毁、类型以及管理方法。

#### 1.蓝图Blueprints
    这些布局允许用户将新的角色无缝集成到模拟中。它们是预先制作好的模型，带有动画和一系列属性。其中一些属性可修改，而另一些则不可修改。
    这些属性包括但不限于车辆颜色、激光雷达传感器的通道数、步行者的速度等等。可用的蓝图及其属性列在蓝图库中。
    车辆和步行者的蓝图都有一个“代数”属性，用于指示它们是新版本（第二代）还是旧版本（第一代）。

##### 1.1管理蓝图库


In [ ]:
import carla
from numpy import random
client = carla.Client('localhost', 2000)
world = client.get_world()
# carla.BlueprintLibrary 类包含一个 carla.ActorBlueprint 元素列表。只有世界对象才能提供对该列表的访问权限。
blueprint_library = world.get_blueprint_library()
# 蓝图都有一个ID来识别它们以及使用该ID生成的角色。可以读取蓝图库来查找特定的ID，随机选择一个蓝图，或者使用通配符模式筛选结果。
# 找到具体的蓝图。
collision_sensor_bp = blueprint_library.find('sensor.other.collision')
# 随机选择一辆车辆蓝图。
vehicle_bp = random.choice(blueprint_library.filter('vehicle.*.*'))
# 除此之外，每个 carla.ActorBlueprint 都有一系列可以获取和设置的 carla.ActorAttribute。
is_bike = [vehicle_bp.get_attribute('number_of_wheels') == 2]
if is_bike:
    vehicle_bp.set_attribute('color', '255,0,0')
# 属性具有一个名为 `carla.ActorAttributeType` 的变量，它从枚举列表中指定属性类型。此外，可修改属性还附带一个推荐值列表。
vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
for attr in vehicle_bps:
    if attr.is_modifiable:
        attr.set_attribute(attr.id, random.choice(attr.recommended_values))

##### 1.2 Actor的生命周期

In [ ]:
from carla import Transform
# 生成,位置和角度
transform = Transform(carla.libcarla.Location(x=230, y=195, z=40), carla.libcarla.Rotation(yaw=180))
actor = world.spawn_actor(vehicle_bp, transform)

# map.get_spawn_points() 用于获取载具的生成点。返回推荐的生成点列表。
spawn_points = world.get_map().get_spawn_points()

# `world.get_random_location()` 方法用于设置步行者的目标位置。该方法返回人行道上的一个随机点。
spawn_point = carla.Transform()
spawn_point.location = world.get_random_location_from_navigation()

# 将相机刚性附加到车辆上
vehicle_bps = world.get_blueprint_library().filter('*vehicle*')
vehicle_bp = random.choice(vehicle_bps)
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
# 检索旁观者对象
spectator = world.get_spectator()
# 通过变换获取观众的位置和旋转。
transform = spectator.get_transform()
camera = world.spawn_actor(camera_bp, transform, attach_to=vehicle_bp)


# 世界对象生成后，会将角色添加到列表中。该列表可以轻松地进行搜索或遍历。
actor_list = world.get_actors()
# 通过id查找actor.
actor = actor_list.find('actor_id')
# 打印出全世界所有限速标志的位置。
for speed_sign in actor_list.filter('traffic.speed_limit.*'):
    print(speed_sign.get_location())

# get() 和 set() 方法，用于管理地图上的actor
print(actor.get_acceleration())
print(actor.get_velocity())

location = actor.get_location()
location.z += 10.0
actor.set_location(location)

# 可以禁用角色的物理效果，使其静止不动。
actor.set_simulate_physics(False)

# Python 脚本执行完毕后，Actor 对象不会被销毁。它们必须显式地自行销毁。
destroyed_sucessfully = actor.destroy() # 如果成功，则返回 True

##### 1.3 Actor的类型

    传感器Sensors

In [ ]:
# 生成一个摄像头传感器，将其连接到车辆上，并指示摄像头将生成的图像保存到磁盘。
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, transform, attach_to=vehicle_bp)
camera.listen(lambda image: image.save_to_disk('output/%06d.png' % image.frame))

    观察者Spectator

In [ ]:
# 移动旁观者角色，使其视角指向目标车辆
spectator = world.get_spectator()
transform = vehicle_bp.get_transform()
spectator.set_transform(carla.Transform(transform.location + carla.Location(z=50),
carla.Rotation(pitch=-90)))

    交通标志和交通信号灯
    目前，CARLA 中只有停车标志、让行标志和交通信号灯被视为 Actor。
    其余的 OpenDRIVE 标志可以通过 API 以 carla.Landmark 的形式访问。
    虽然可以通过这些实例访问它们的信息，但它们在模拟中并不作为 Actor 存在。

In [ ]:
# 让交通信号灯影响车辆
if vehicle_actor.is_at_traffic_light():
    traffic_light = vehicle_actor.get_traffic_light()
# 将红灯改为绿灯
if traffic_light.get_state() == carla.TrafficLightState.Red:
    traffic_light.set_state(carla.TrafficLightState.Green)
    traffic_light.set_set_green_time(4.0)

    车辆Vehicles

In [ ]:
# carla.Vehicle 是一种特殊的 Actor。它包含特殊的内部组件，用于模拟轮式车辆的物理特性。这是通过应用四种不同的控件来实现的：
# carla.VehicleControl 提供驾驶指令的输入，例如油门、转向、刹车等。

# carla.VehiclePhysicsControl 定义了车辆的物理属性，并包含两个控制器：
# carla.GearPhysicsControl，用于控制配置变速箱/离合器/换挡逻辑的物理参数。
# carla.WheelPhysicsControl，用于对每个车轮进行特定控制。
vehicle.apply_physics_control(carla.VehiclePhysicsControl(max_rpm = 5000.0, center_of_mass = carla.Vector3D(0.0, 0.0, 0.0), torque_curve=[[0,400],[5000,400]]))

# 车辆由一个 carla.BoundingBox 包裹。该边界框允许对车辆应用物理效果，并实现碰撞检测。
box = vehicle.bounding_box
print(box.location)         # 相对于车辆的位置。
print(box.extent)           # XYZ 半盒尺寸（单位：米）。

# 启用“扫描车轮碰撞”参数可以改善车辆车轮的物理特性。
# 默认的车轮物理特性使用从车轮轴线到地面的单条射线进行碰撞检测，但启用“扫描车轮碰撞”后，将对车轮的整个体积进行碰撞检测。启用方法如下：
physics_control = vehicle.get_physics_control()
physics_control.use_sweep_wheel_collision = True
vehicle.apply_physics_control(physics_control)

# 车辆还包含其他一些独有的功能：
# 自动驾驶模式会将车辆连接到交通管理系统，以模拟真实的城市路况。该模块采用硬编码，并非基于机器学习。
vehicle.set_autopilot(True)

# 车辆灯光需要用户手动开启和关闭。每辆车都有一组灯光状态记录在 `carla.VehicleLightState` 中
# 可以使用 `carla.Vehicle.get_light_state` 和 `carla.Vehicle.set_light_state` 方法随时获取和更新车辆灯光状态
# 打开示位灯
current_lights = carla.VehicleLightState.NONE
current_lights |= carla.VehicleLightState.Position
vehicle.set_light_state(current_lights)

    行人Walkers
    carla.WalkerControl 控制行人以特定方向和速度移动，并允许其跳跃。
    行人可以由 AI 控制，但没有自动驾驶模式。carla.WalkerAIController 对象围绕其所依附的对象移动。

In [ ]:
# 每个AI控制器都需要初始化、设定目标，以及（可选）设定速度。停止控制器的操作方式相同。
# 要摧毁 AI 行人，停止 AI 控制器，然后摧毁执行者和控制器。
ai_controller.start()
ai_controller.go_to_location(world.get_random_location_from_navigation())
ai_controller.set_max_speed(1 + random.random())  # Between 1 and 2 m/s (default is 1.4 m/s).
...
ai_controller.stop()